In [45]:
import pandas as pd
from division_aggregation_function import division_aggregation

In [46]:
index = pd.read_csv('../data/google_data/index.csv')
epi = pd.read_csv('../data/google_data/epidemiology.csv')

In [47]:
territories = ['US', 'US_AS', 'US_GU', 'US_MP', 'US_PR', 'US_VI']
location_keys = index[index.country_code == 'US']['location_key']
# select only us locations and drop the terriroties to leave only states and DC
location_keys = location_keys[[('US' in s and len(s) <= 5) for s in location_keys]][~location_keys.isin(territories)]

In [48]:
# filter all dataframes to only include US states and DC
epi = epi[epi.location_key.isin(location_keys)]
# make divisional region column
epi['region'] = epi["location_key"].apply(division_aggregation)

In [50]:
# drop all columns except for date, location_key, and new_confirmed
epi_subset = epi.set_index(['date', 'location_key', 'region'])['new_confirmed'].reset_index()

In [51]:
earliest_date = epi_subset[epi_subset["new_confirmed"] > 0]["date"].min()

In [52]:
epi_subset = epi_subset[epi_subset["date"] >= earliest_date]

In [53]:
epi_subset.head()

,date,location_key,region,new_confirmed
0,2020-01-22,US_AK,Pacific,0.0
1,2020-01-23,US_AK,Pacific,0.0
2,2020-01-24,US_AK,Pacific,0.0
3,2020-01-25,US_AK,Pacific,0.0
4,2020-01-26,US_AK,Pacific,0.0


In [55]:
epi_subset.to_csv('../data/univariate_data.csv', index=False)

In [59]:
epi_subset.shape

(49372, 4)

In [60]:
# make regional data univariate data 
regional_epi_subset = epi_subset.groupby(["date", "region"]).sum().reset_index()
regional_epi_subset.head()

,date,region,new_confirmed
0,2020-01-19,Pacific,1.0
1,2020-01-20,Pacific,0.0
2,2020-01-21,Pacific,1.0
3,2020-01-22,East North Central,0.0
4,2020-01-22,East South Central,0.0


In [62]:
regional_epi_subset.to_csv('../data/regional_univariate_data.csv', index=False)